# MongoDB Atlas vector-search with SuperDuperDB

In [ ]:
!pip install superduperdb

In [ ]:
from superduperdb.db.base.build import build_datalayer
from superduperdb import CFG
import os

ATLAS_URI = "mongodb+srv://<user>:<password>@<host>/?retryWrites=true&w=majority/my_database"
OPENAI_API_KEY = "<OPEN-API-KEY>"

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

CFG.data_backend = ATLAS_URI
CFG.vector_search = ATLAS_URI

db = build_datalayer()

In [ ]:
!curl -O https://superduperdb-public.s3.eu-west-1.amazonaws.com/pymongo.json

In [ ]:
import json
from superduperdb.db.mongodb.query import Collection
from superduperdb.container.document import Document as D

with open('pymongo.json') as f:
    data = json.load(f)

db.execute(Collection('documents').insert_many([D(r) for r in data]))

In [ ]:
db.show('vector_index')

In [ ]:
from superduperdb.container.vector_index import VectorIndex
from superduperdb.container.listener import Listener
from superduperdb.ext.numpy.array import array
from superduperdb.ext.openai.model import OpenAIEmbedding

model = OpenAIEmbedding(model='text-embedding-ada-002')

db.add(
    VectorIndex(
        identifier=f'pymongo-docs',
        indexing_listener=Listener(
            model=model,
            key='value',
            select=Collection('documents').find(),
            predict_kwargs={'max_chunk_size': 1000},
        ),
    )
)

In [ ]:
from superduperdb.db.mongodb.query import Collection
from superduperdb.container.document import Document as D
from IPython.display import *

query = 'Query the database'

result = db.execute(
    Collection('documents')
        .like(D({'value': query}), vector_index='pymongo-docs', n=5)
        .find()
)

for r in result:
    display(Markdown(f'### `{r["parent"] + "." if r["parent"] else ""}{r["res"]}`'))
    display(Markdown(r['value']))